# Genera subtítulos a un video y traducelos a otro idioma ⏯️ 🍿.

En este notebook aprenderás a extraer subitulos de un video y a generar unos nuevos en otro idioma. 


## Este ejemplo vas a usar 2servicios de AI/ML que puedes emplear como llamadas de API: 

- [Amazon Transcribe](https://aws.amazon.com/es/transcribe/): un servicio de reconocimiento automático de voz que utiliza modelos de aprendizaje automático para convertir audio en texto. Puede usarlo Amazon Transcribe como un servicio de transcripción independiente o para añadir speech-to-text capacidades a cualquier aplicación. . [Más información](https://docs.aws.amazon.com/es_es/transcribe/latest/dg/what-is.html)

- [Amazon Translate](https://aws.amazon.com/translate/): un servicio de traducción de textos que utiliza tecnologías avanzadas de aprendizaje automático para ofrecer traducciones de alta calidad a pedido. Puede usar Amazon Translate para traducir documentos de texto no estructurados o para crear aplicaciones que funcionen en varios idiomas. ConsulteIdiomas y códigos de idioma admitidospara obtener información sobre los idiomas que admite Amazon Translate. [Más información](https://docs.aws.amazon.com/es_es/translate/latest/dg/what-is.html)

### Requisitos:
- [Una cuenta de AWS](https://aws.amazon.com/es/free/)
- [Foundational knowledge of Python](https://community.aws/tutorials/build-a-knowledge-base-with-multilingual-q-and-a-gen-ai)

### Configuración previa:
- [Uso de AWS Command Line Interface CLI](https://docs.aws.amazon.com/es_es/cli/latest/userguide/cli-chap-welcome.html)
- [Instalar CLI](https://docs.aws.amazon.com/es_es/cli/latest/userguide/getting-started-install.html)
- [Configurar credenciales de AWS para uso local](https://docs.aws.amazon.com/es_es/cli/latest/userguide/cli-configure-files.html)


## ⏯️ 🍿 Empecemos!! 

### Paso 0: [Instala Boto3](https://boto3.amazonaws.com/v1/documentation/api/latest/guide/quickstart.html#install-boto3) [Más información de Boto3](https://aws.amazon.com/es/sdk-for-python/)
Es una librería de python para facilitar la integración con los servicios de AWS (Amazon Web Services)

In [ ]:
### Instala la ultima version de Boto3 via pip: 

!pip install boto3

In [14]:
# Boto3 session with credentials taken from environtment
import boto3
import os

AWS_ACCESS_KEY_ID = YOU_AWS_ACCESS_KEY_ID
AWS_SECRET_ACCESS_KEY = YOU_AWS_SECRET_ACCESS_KEY
AWS_SESSION_TOKEN= YOU_AWS_SESSION_TOKEN
REGION_NAME = YOU_REGION

my_session = boto3.Session(
    aws_access_key_id     = AWS_ACCESS_KEY_ID,
    aws_secret_access_key = AWS_SECRET_ACCESS_KEY,
    aws_session_token     = AWS_SESSION_TOKEN,
    region_name=REGION_NAME
)


### Paso 1: Detecta extrae el texto en formato .srt desde el audio del video. 

> 🚨Nota: Cada servicio tiene un cliente que debes inicializar antes de hacer cada llamada de API. 

Este paso lo realizamos empleando la llamda de API [start_transcription_job](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/transcribe/client/start_transcription_job.html) de [Boto3 Transcribe client](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/transcribe.html).

### Amazon transcribe toma el video desde un bucket de [Amazon S3](https://docs.aws.amazon.com/s3/index.html), por lo que debes crearlo primero y luego subir el video. [Sigue los pasos decomo crear un Bucket de S3 aca](https://docs.aws.amazon.com/es_es/AmazonS3/latest/userguide/creating-bucket.html)

In [15]:
transcribe_client  = my_session.client('transcribe')

def crea_subitulos(jobName,SOURCE_LANG_CODE,s3Path,s3bucket,name_out):

    response = transcribe_client.start_transcription_job(
                    TranscriptionJobName=jobName,
                    LanguageCode= SOURCE_LANG_CODE,
                    MediaFormat='mp4',
                    Media={
                    'MediaFileUri': s3Path
                    },
                    OutputBucketName = s3bucket,
                    OutputKey=name_out, 
                    Subtitles={
                    'Formats': [
                        'srt'
                    ]}
                    )

    TranscriptionJobName = response['TranscriptionJob']['TranscriptionJobName']
        
    print("Processing....")
    print("TranscriptionJobName : {}".format(TranscriptionJobName))

> Lenguajes adminitos con sus códigos [Supported languages](https://docs.aws.amazon.com/es_es/transcribe/latest/dg/supported-languages.html).

### Paso 2: Traduce los subitulos a otro idioma. 
Antes de iniciar este paso debes esperar que los subitulos esten listos, esto dependera de la duración del video, te recomiendo subir videos con tamaño reducido, te importa es la calidad den audio no de la imagen. 

Una vez terminada la transcrición procede a su traducción. 

Este paso lo realizamos empleando la llamda de API [TranslateText](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/comprehend/client/detect_dominant_language.html#detect-dominant-language) de [Boto3 Translate client](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/comprehend.html). 

[supported language codes](https://docs.aws.amazon.com/translate/latest/dg/what-is-languages.html) 

In [16]:
translate_client = my_session.client('translate')

def traducir_subitulos(text,sourceLanguage,targetLanguage):
    translate = translate_client.translate_text(
                Text=text, 
                SourceLanguageCode=sourceLanguage,
                TargetLanguageCode=targetLanguage)
                
    result = translate['TranslatedText']
    return result

> Aca el listado de idiomas manejados por translate y sus códigos [Supported languages](https://docs.aws.amazon.com/translate/latest/dg/what-is-languages.html).

In [17]:
#Primero necesitamos descargar el archivo para procesarlo.
#y lo hacemos con esta funcion.
client_s3 = my_session.client('s3')

def download_file(bucket, key, filename):
    with open(filename, "wb") as data:
        client_s3.download_fileobj(bucket, key, data)
    print("Download file from s3://{}{}".format(bucket,key))
    return True

In [18]:
#Por ser un archivo .srt se debe procesar por linea, lo cual se hará con la siguiente función. 
def procesa_traduccion(sourceLanguage,targetLanguage,s3object,s3bucket, filename):
    if(filename.endswith("srt")): 
        srt_file_out = targetLanguage + "_" + filename

        download_file(s3bucket, s3object, filename)

        with open(filename) as f:
            str = f.readlines()

        #Los subtítulos se repiten cada 4 líneas empezando por la segunda. 
        #Taking ten en cuenta que la primera línea es 0.

        sub=[]
        for n in range (2,len(str),4):
            sub.append(n)
            traslate_out=[]

        n=0
        for line in str:
            if n in sub: #Only the lines that correspond to the subtitle text are translated.
                lines_traslate=traducir_subitulos(line,sourceLanguage,targetLanguage)
            else:
                lines_traslate = line
            traslate_out.append(lines_traslate)
            n=n+1
            
        with open(srt_file_out, 'w') as temp_file:
            for item in traslate_out:
                temp_file.write(item)
        
            
    else:
        print("No es .srt")
            
        print("Listo!")

### Paso 3: Creemos subitulos traducidos!

In [19]:
jobName = "Demojob_name"
SOURCE_LANG_CODE = "en-US"
s3Path = "s3://demo-eli-us-east-1/demo_subitulo_video/demo_subtitulos.mp4" #s3://DOC-EXAMPLE-BUCKET/my-media-file.mp3
bucket = "demo-eli-us-east-1"
name_out = "subtitulo_listo"
crea_subitulos(jobName,SOURCE_LANG_CODE,s3Path,bucket,name_out)

Processing....
TranscriptionJobName : Demojob_name


In [20]:
#Para ver el estadod el job de transcribe
response = transcribe_client.get_transcription_job(
    TranscriptionJobName=jobName
)
response

{'TranscriptionJob': {'TranscriptionJobName': 'Demojob_name',
  'TranscriptionJobStatus': 'IN_PROGRESS',
  'LanguageCode': 'en-US',
  'MediaFormat': 'mp4',
  'Media': {'MediaFileUri': 's3://demo-eli-us-east-1/demo_subitulo_video/demo_subtitulos.mp4'},
  'Transcript': {},
  'StartTime': datetime.datetime(2023, 8, 24, 12, 53, 34, 687000, tzinfo=tzlocal()),
  'CreationTime': datetime.datetime(2023, 8, 24, 12, 53, 34, 653000, tzinfo=tzlocal()),
  'Settings': {'ChannelIdentification': False, 'ShowAlternatives': False},
  'Subtitles': {'Formats': ['srt'], 'SubtitleFileUris': []}},
 'ResponseMetadata': {'RequestId': '5bc15908-b1a7-4b8b-b686-e2f19d6737d6',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '5bc15908-b1a7-4b8b-b686-e2f19d6737d6',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '432',
   'date': 'Thu, 24 Aug 2023 16:53:37 GMT'},
  'RetryAttempts': 0}}

In [ ]:
response['TranscriptionJob']['Subtitles']['SubtitleFileUris'][0].split("/")

In [ ]:
s3bucket =response['TranscriptionJob']['Subtitles']['SubtitleFileUris'][0].split("/")[3]
s3object = response['TranscriptionJob']['Subtitles']['SubtitleFileUris'][0].split("/")[4]
filename = response['TranscriptionJob']['Subtitles']['SubtitleFileUris'][0].split("/")[4]
targetLanguage = "es" 
sourceLanguage = "en"
procesa_traduccion(sourceLanguage,targetLanguage,s3object,s3bucket, filename)

## ¡Gracias!

Te dejo mis redes: 
🇻🇪🇨🇱 [Dev.to](https://dev.to/elizabethfuentes12) [Linkedin](https://www.linkedin.com/in/lizfue/) [GitHub](https://github.com/elizabethfuentes12/) [Twitter](https://twitter.com/elizabethfue12) [Instagram](https://www.instagram.com/elifue.tech) [Youtube](https://www.youtube.com/channel/UCr0Gnc-t30m4xyrvsQpNp2Q)
[linktr](https://linktr.ee/elizabethfuentesleone)